In [1]:

from minigrid.wrappers import ImgObsWrapper
from multigrid.wrappers import MultiAgentImgObsWrapper
from stable_baselines3 import PPO
import torch.nn as nn

import torch
import gymnasium as gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
# from features_extractor import MinigridFeaturesExtractor
from typing import Dict
from multigrid.envs import EmptyEnv




pygame 2.5.1 (SDL 2.28.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class MinigridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 64, normalized_image: bool = False) -> None:
        super().__init__(observation_space, features_dim)
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (2, 2)),
            nn.ReLU(),
            nn.Flatten(),
        )

        print(observation_space)
        
        n = observation_space.shape[0]
        m = observation_space.shape[1]
        print(n,m)
        self.image_embedding_size = ((n-1)//2-2)*((m-1)//2-2)*64


        # Compute shape by doing one forward pass
        tens= torch.as_tensor(observation_space.sample()[None]).to(torch.uint8).float()
        print(tens.shape)
        tens = torch.tensor(tens).float().permute(0,3,1,2)
        with torch.no_grad():
            n_flatten = self.cnn(tens).shape[1]
        print(features_dim, self.image_embedding_size)
        lin = nn.Linear(n_flatten, features_dim)
        self.linear = nn.Sequential(lin, nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        
        print(f"from_forward{observations.shape}")
        observations = torch.Tensor(observations)
        return self.linear(self.cnn(observations))

In [3]:


policy_kwargs = dict(
    features_extractor_class=MinigridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)

env = EmptyEnv(render_mode="rgb_array", agents=2)
env = MultiAgentImgObsWrapper(env)
# env = ImgObsWrapper(env)



# model = PPO("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1)
# model.learn(20)

In [4]:
obs = env.reset()
# obs = torch.tensor(obs[0]).unsqueeze(0)
# obs.shape


In [5]:
env.observation_space

{0: Box(0, 255, (7, 7, 3), int64), 1: Box(0, 255, (7, 7, 3), int64)}

In [6]:
from multigrid.envs import EmptyEnv
from multigrid.wrappers import MultiAgentImgObsWrapper

env2 = EmptyEnv(render_mode="rgb_array", agents=2)
env2 = MultiAgentImgObsWrapper(env2)

obs2 = env.reset()
obs2 = torch.tensor(obs[0][0]).unsqueeze(0)
print("asdas")
feature_extractor  = MinigridFeaturesExtractor(env2.observation_space[0], features_dim=64)


: 

In [7]:
obs = env.reset()
obs = torch.tensor(obs[0][0]).unsqueeze(0)
print("asdas")
feature_extractor  = MinigridFeaturesExtractor(env.observation_space[0], features_dim=64)

# for agent_obs in obs:
#     agent_obs =torch.tensor(agent_obs).float().permute(0,3,1,2)

# x = feature_extractor(obs)
# obs, _ , _, _, _ = env.step(env.action_space.sample())
# obs =torch.tensor(obs).float().unsqueeze(0).permute(0,3,1,2)
# x = feature_extractor(obs)

: 

In [7]:
env.observation_space[0]

Box(0, 255, (7, 7, 3), int64)

In [8]:
obs = env.reset()
obs = torch.tensor(obs[0][0]).unsqueeze(0)

feature_extractor  = MinigridFeaturesExtractor(env.observation_space[0], features_dim=64)

# for agent_obs in obs:
#     agent_obs =torch.tensor(agent_obs).float().permute(0,3,1,2)

: 